In [9]:
import json,os,glob

import torch

In [12]:

save_name = 'prediction.txt'


BASE = '/home/ubuntu/sn15_share_dir/av_deepfake/train_results/exp12/deepfake_lavdf_emo_exp12_2020_12_06_07_17_03'
result = []
exist_video = set()
for subi in range(1,8):
    json_folder = os.path.join(BASE, f'{subi}')
    # json_folder = '/home/ubuntu/sn15_share_dir/av_deepfake/train_results/exp4/deepfake_byola_exp4_2020_11_14_13_30_32'
    json_files = glob.glob(os.path.join(json_folder,'*.json'))
    for json_file in json_files:
        # print(json_file)
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        for item in data:
            video_id = item['video_id']
            if video_id not in exist_video:
                exist_video.add(video_id)
            else:
                continue
            video_cls = item['video_cls']
            prob = torch.sigmoid(torch.tensor(video_cls)).item()
            if prob>0.9: 
                prob = 1.0
            result.append([video_id,str(prob)])
result = sorted(result, key=lambda x: x[0])

with open(os.path.join(BASE, save_name), 'w') as fo:
    fo.write('\n'.join([';'.join(i) for i in result]))
    

In [13]:

"""
{
    "000001.mp4": [
        [0.9541, 4.0, 4.8], // [confidence to be fake, start time, end time]
        [0.2315, 4.24, 5.52],
        [0.0012, 0.48, 2.6],
        [0.0002, 6.56, 8.8],
        ...
    ],
    ...
}
"""
save_name2 = 'prediction.json'

BASE = '/home/ubuntu/sn15_share_dir/av_deepfake/train_results/exp12/deepfake_lavdf_emo_exp12_2020_12_04_10_28_16'

result = {}
exist_video = set()
# with open(os.path.join(BASE, save_name2),'a+') as fid:
for subi in range(1,8):
    json_folder = os.path.join(BASE, f'{subi}')
    # json_folder = '/home/ubuntu/sn15_share_dir/av_deepfake/train_results/exp4/deepfake_byola_exp4_2020_11_14_13_30_32'
    json_files = glob.glob(os.path.join(json_folder,'data*.json'))
    for json_file in json_files:
        
        with open(json_file, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(json_file)
                print(e)
        for item in data:
            tmp_result = []
            video_id = item['video_id']
            if video_id not in exist_video:
                exist_video.add(video_id)
            else:
                continue
            segments = item['segments']
            for i,score in enumerate(item['scores']):
                if score>0.2:
                    tmp_result.append([score, segments[i][0], segments[i][1]])
            if len(tmp_result)==0: 
                tmp_result.append([0,0,0])
            result[video_id] = tmp_result 


with open(os.path.join(BASE,save_name2), 'w', encoding='utf-8') as f:
    json.dump(result, f, sort_keys=True,ensure_ascii=False, indent=4)
    
                        
